In [ ]:
!pip install ultralytics
import ultralytics

In [16]:
import numpy as np
import pandas as pd
import random
import tqdm
from PIL import Image

Format CNR-EXT dataset to match YOLO requirements

In [15]:
# Rescale and normalize bounding boxes
rescale_x, rescale_y = 1/2582, 1/1944
boxes_combined = pd.DataFrame()
for i in range(1, 10):
    boxes = pd.read_csv(f"../bounding_boxes/camera{i}.csv")
    boxes['X'] = rescale_x * boxes['X']
    boxes['W'] = rescale_x * boxes['W']
    boxes['Y'] = rescale_y * boxes['Y']
    boxes['H'] = rescale_y * boxes['H']
    boxes['C'] = f"C0{i}"
    boxes.to_csv(f"../bounding_boxes/camera{i}_rescaled.csv")
    boxes_combined = boxes_combined.append(boxes)

boxes_combined.to_csv(f"../bounding_boxes/boxes_combined.csv")


In [35]:
def get_slot_ids(file, weather, date, time, camera):
    slot_ids = []
    with open(file, 'r') as file:
        for line in file:
            real_img_path, real_lbl = line.split(' ')
            sections = real_img_path.split('/')
            camera_q, slot = sections[-1].split('_')[-2:]
            slot = slot[0:3]
            # camera - C06, slot - 240
            weather_q, date_q, time_q = sections[0], sections[1], sections[-1].split('.')[0:2]
            time_q = time_q[0][-2:] + time_q[1][0:2]
            if(weather == weather_q and date == date_q 
               and time == time_q and camera == camera_q):
                slot_ids.append(slot)
    return slot_ids



labels_dir = "../LABELS"
data_files = ["train", "val"]
boxes_combined = pd.read_csv('../bounding_boxes/boxes_combined.csv')
completed_configs = {}
for i, mode in enumerate(data_files):
    cnt = 0
    data_file = f"{labels_dir}/{mode}.txt"
    with open(data_file, 'r') as file:
        for line in file:
            cnt += 1
            img_path, lbl_path = f"../dataset/{mode}/images/{cnt:06d}.png", f"../dataset/{mode}/labels/{cnt:06d}.txt"
            real_img_path, real_lbl = line.split(' ')
            sections = real_img_path.split('/')
            camera, slot = sections[-1].split('_')[-2:] # camera - C06, slot - 240
            slot = slot[0:3]
            weather, date, time = sections[0], sections[1], sections[-1].split('.')[0:2]
            time = time[0][-2:] + time[1][0:2] # weather - SUNNY, date - 2015-11-12, time - 0709

            key = f"{weather}/{date}/{camera}/{time}"
            if key not in completed_configs:
                slot_ids = get_slot_ids(data_file, weather, date, time, camera)
                completed_configs[key] = 1
            else:
                print("Key exists!")
                continue
            yolo_label = []
            for slot_id in slot_ids:
                box = boxes_combined[(boxes_combined['C'] == camera) & 
                                     (boxes_combined['SlotId'] == int(slot_id))]
                if box.shape[0] == 1:
                    box = box.iloc[0]  # Extract the single row as a Series
                elif box.shape[0] > 1:
                    raise ValueError(f"Multiple rows found for camera={camera} and slot_id={slot_id}")
                else:
                    raise ValueError(f"No matching row found for camera={camera} and slot_id={slot_id}")
                X, Y, W, H = box['X'], box['Y'], box['W'], box['H']
                yolo_label.append(real_lbl.rstrip('\n') + " " + str(X) + " " + str(Y) + " " + str(W) + " " + str(H) + '\n')
            yolo_label = ''.join(yolo_label)

            with open(lbl_path, 'w') as f:
                f.write(yolo_label)

            yolo_image = Image.open(f"../FULL_IMAGE_1000x750/{weather}/{date}/camera{camera[-1]}/{date}_{time}.jpg")
            print(img_path)
            yolo_image.save(img_path)

../dataset/train/images/000001.png
../dataset/train/images/000002.png
../dataset/train/images/000003.png
../dataset/train/images/000004.png
../dataset/train/images/000005.png
../dataset/train/images/000006.png
../dataset/train/images/000007.png
../dataset/train/images/000008.png
../dataset/train/images/000009.png
../dataset/train/images/000010.png
../dataset/train/images/000011.png
../dataset/train/images/000012.png
../dataset/train/images/000013.png
../dataset/train/images/000014.png
../dataset/train/images/000015.png
../dataset/train/images/000016.png
../dataset/train/images/000017.png
../dataset/train/images/000018.png
../dataset/train/images/000019.png
../dataset/train/images/000020.png
../dataset/train/images/000021.png
../dataset/train/images/000022.png
../dataset/train/images/000023.png
../dataset/train/images/000024.png
../dataset/train/images/000025.png
../dataset/train/images/000026.png
../dataset/train/images/000027.png
../dataset/train/images/000028.png
../dataset/train/ima